In [ ]:
import pandas as pd
import numpy as np
import os
import csv
import joblib

In [ ]:
# value에 ","가 들어간 csv 파일 재구축
file_path = '/ml/algorithmlabs.csv/crawldata/20230203/ALGORITHM.ALAB_STDSCHOOLSTS.csv'
f = open(file_path, encoding='utf-8')
reader = csv.reader(f)
csv_list = []
for i in reader : 
    csv_list.append(i)
f.close()

# 0207 수정
long = len(csv_list[0])
for i in range(len(csv_list)) : 
    idx = len(csv_list[i]) - long
    del csv_list[i][3:4+idx]

In [ ]:
df_1 = pd.DataFrame(csv_list[1:], columns = csv_list[0])
df_1 = df_1.replace('', np.nan)

In [ ]:
# value에 ","가 들어간 csv 파일 재구축
file_path = '/ml/algorithmlabs.csv/crawldata/20230203/ALGORITHM.ALAB_STDENTER.csv'
f = open(file_path, encoding='utf-8')
reader = csv.reader(f)
csv_list = []
for i in reader : 
    csv_list.append(i)
f.close()

# 0207 수정
long = len(csv_list[0])
for i in range(len(csv_list)) : 
    idx = len(csv_list[i]) - long
    del csv_list[i][3:4+idx]

In [ ]:
df_2 = pd.DataFrame(csv_list[1:], columns = csv_list[0])
df_2 = df_2.replace('', np.nan)

In [ ]:
date = '20230203'

In [ ]:
# 원본 데이터 읽기
ests = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/ALGORITHM.ALAB_STDCHANGESTS.csv')
cla = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/ALGORITHM.ALAB_STDCLASS.csv')
fee = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/ALGORITHM.ALAB_STDFEE.csv')
fees = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/ALGORITHM.ALAB_STDFEE_SUB.csv')
grade = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/ALGORITHM.ALAB_STDGRADE.csv')
gradet= pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/ALGORITHM.ALAB_STDGRADE_TOTAL.csv')
prize = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/ALGORITHM.ALAB_STDPRIZE.csv')
scholar = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/ALGORITHM.ALAB_STDSCHOLAR.csv')
gate = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/GATEDB.dbo.INNOV_ENTER_COUNT.csv')
sts = df_1.copy()
enter = df_2.copy()

pro = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.PROGRAM_PER.csv')
get = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.GETPOINT_PER.csv')
vmi = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.VMILEAGE_PER_CHECK.csv')
pau = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.ICAP_PAU.csv')
jtu = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.ICAP_JTU.csv')
ncs = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.ICAP_NCS.csv')
hcp = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.ICAP_HCP.csv')
dlc = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.ICAP_DLC.csv')
counsel = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.ICAP_COUNSEL.csv')

book = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/SOLARS8.INNOV_CHARGE_BOOK.csv')
room = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/SOLARS8.INNOV_ROOM_CHARGE.csv')
seat = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/SOLARS8.INNOV_SEAT_CHARGE.csv')

column_name = pd.read_csv('/ml/temp_wg/develop/feature_korean_name.csv')
# 0207 수정
name_data = pd.read_csv('/ml/algorithmlabs.csv/crawldata/'+date+'/h_mju.VMILEAGE_PER_CHECK.csv')

In [ ]:
# 기준년도, 기준학기를 만들기 위한 함수
def create_ys(year, semester) : 
    bsts_1 = enter.copy()
    bsts_1['CHG_YEAR'] = year
    bsts_1['CHG_SMT'] = semester
    bsts_1 = bsts_1[['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT', 'ENTER_YEAR', 'ENTER_SMT', 'JUN_NM', 'ENTER_CHASU', 'ENTER_PART_RANK', 'HI_SCH_AREA', 'GRADUATION_YEAR', 'ENTER_TYPE']]
    return bsts_1

In [ ]:
# 특정년도, 특정학기에 받은 장학금 총 횟수, 누적금액을 계산하기 위한 파생변수 생성 함수
def scholar_sum(year, semester) : 
    fee_201 = en_scholar[(en_scholar['SCHOLAR_YEAR']==year) & (en_scholar['SCHOLAR_SMT']==semester)].sort_values(by='STUDENT_CD', ascending=True)
    fee_201_sum = []
    fee_201_cnt = []

    for id in fee_201['STUDENT_CD'].unique().tolist() :
        cnt = fee_201[fee_201['STUDENT_CD']==id]['SCHOLAR_AMT'].sum()
        count = len(fee_201[fee_201['STUDENT_CD']==id])
        for i in range(len(fee_201[fee_201['STUDENT_CD']==id])) : 
            fee_201_sum.append(cnt)
            fee_201_cnt.append(count)
            
    fee_201['SCHOLAR_CNT'] = fee_201_cnt 
    fee_201['SCHOLAR_SUM'] = fee_201_sum
    
    return fee_201

In [ ]:
# 도서관 데이터 구축을 위한 함수
def lib(data, year, semester, column_name) : 
    temp = data[(data['CHG_YEAR']==year) & (data['CHG_SMT']==semester)]
    new_data = []
    
    stu_id = temp['STUDENT_CD'].unique().tolist()
    for name in stu_id : 
        stu_data = temp[temp['STUDENT_CD']==name]
        enter_values = stu_data[column_name].sum()
        new_data.append([name, enter_values, year, semester])
        
    result = pd.DataFrame(new_data, columns = temp.columns.tolist())
    return result

In [ ]:
# stdenter
en_enter = pd.concat([create_ys(2020, 1), create_ys(2020, 2), create_ys(2021, 1), create_ys(2021, 2), create_ys(2022, 1), create_ys(2022, 2)], axis=0)
en_enter.replace({'ENTER_SMT' : {'1학기' : 1}}, inplace=True)
en_enter['ENTER_YEAR'] = en_enter['ENTER_YEAR'].astype('int64')
df = en_enter.loc[en_enter['CHG_YEAR']>=en_enter['ENTER_YEAR']]
df.drop(['ENTER_SMT'], axis=1, inplace=True)

In [ ]:
# schoolsts
en_sts = sts.copy()
en_sts = en_sts[['STUDENT_CD', 'BIRTH', 'SEX', 'CAMPUS_NM', 'DEPT_NM', 'MAJOR_NM', 'DOUBLE_MAJOR_NM', 'LINK_MAJOR_NM', 'SUB_MAJOR_NM']]
df = pd.merge(df, en_sts, on = ['STUDENT_CD'], how='left')

In [ ]:
df['STUDENT_CD'] = df['STUDENT_CD'].astype('int64')

In [ ]:
# stdchangests
en_ests = ests.copy()
sorting = en_ests.sort_values(by='CHG_DT', ascending=True).reset_index(drop=True)
en_ests = sorting.drop_duplicates(['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], keep='last')
en_ests = en_ests[['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT', 'CHG_DIV', 'CHG_NM', 'CHG_REA']]
en_ests.replace({'CHG_SMT' : {'1학기' : 1, '2학기' : 2}}, inplace=True)
df = pd.merge(df, en_ests, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# 제적되어 다음학기에 생성불가능한 row들을 사전제거
df = df.sort_values(by=['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], ascending=True).reset_index(drop=True)
drop_idx = []
for name in df['STUDENT_CD'].unique().tolist() : 
    tp = df[df['STUDENT_CD']==name]
    if '제적' in tp['CHG_DIV'].tolist() : 
        idx = tp['CHG_DIV'].tolist().index('제적')
        if (idx+1 != len(tp)) : 
            for i in range(idx+1, len(tp)) : 
                drop_idx.append(tp.index.tolist()[i])
                
df.drop(drop_idx, axis=0, inplace=True)

In [ ]:
# stdfee
fee = fee.drop_duplicates(['STUDENT_CD', 'REG_YEAR', 'REG_SMT'], keep='last').dropna(subset='REG_DATE', axis=0)
en_fee = fee[['STUDENT_CD', 'REG_YEAR', 'REG_SMT', 'IS_REG_STUDENT_FEE', 'IS_REG_DIV', 'IS_REG_YET']]
en_fee.replace({'REG_SMT' : {10 : 1, 15 : 1.5, 20 : 2, 25 : 2.5}}, inplace=True)
en_fee.rename(columns={'REG_YEAR' : 'CHG_YEAR', 'REG_SMT' : 'CHG_SMT'}, inplace=True)
df = pd.merge(df, en_fee, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# stdfee_sub
en_fees = fees.replace({'REG_SMT' : {10 : 1, 20 : 2}})
en_fees.rename(columns={'REG_YEAR' : 'CHG_YEAR', 'REG_SMT' : 'CHG_SMT'}, inplace=True)
df = pd.merge(df, en_fees, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# stdscholar
en_scholar = scholar.loc[scholar['SCHOLAR_YEAR']>2019]
en_scholar.replace({'SCHOLAR_SMT' : {'1학기' : 1, '2학기' : 2}}, inplace=True)
en_scholar = en_scholar.astype({'STUDENT_CD' : 'int64'})

sch_1 = scholar_sum(2020, 1)
sch_2 = scholar_sum(2020, 2)
sch_3 = scholar_sum(2021, 1)
sch_4 = scholar_sum(2021, 2)
sch_5 = scholar_sum(2022, 1)
sch_6 = scholar_sum(2022, 2)

sum_scholar = pd.concat([sch_1, sch_2, sch_3, sch_4, sch_5, sch_6], axis=0)
sum_scholar = sum_scholar[['STUDENT_CD', 'SCHOLAR_YEAR', 'SCHOLAR_SMT', 'SCHOLAR_CNT', 'SCHOLAR_SUM', 'CONFIRM_YN', 'CONTINUE_YN', 'WORK_YN']]
sum_scholar = sum_scholar.drop_duplicates(['STUDENT_CD', 'SCHOLAR_YEAR', 'SCHOLAR_SMT'], keep='last')
sum_scholar.rename(columns={'SCHOLAR_YEAR' : 'CHG_YEAR', 'SCHOLAR_SMT' : 'CHG_SMT'}, inplace=True)
df = pd.merge(df, sum_scholar, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# stdclass
en_cla = cla.copy()
en_cla.replace({'CLASS_SMT' : {'1학기' : 1, '2학기' : 2}}, inplace=True)
en_cla = en_cla.drop(['CDTTOTAL'], axis=1)
en_cla.rename(columns={'CLASS_YEAR' : 'CHG_YEAR', 'CLASS_SMT' : 'CHG_SMT'}, inplace=True)
df = pd.merge(df, en_cla, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# stdgrade
en_grade = grade.copy()
en_grade.replace({'GRADE_SMT' : {'1학기' : 1, '2학기' : 2}}, inplace=True)
en_grade.rename(columns={'GRADE_YEAR' : 'CHG_YEAR', 'GRADE_SMT' : 'CHG_SMT'}, inplace=True)
df = pd.merge(df, en_grade, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# total gpa 파생변수 생성
total_gpa = [] 

for name in df['STUDENT_CD'].unique().tolist() : 
    tp = df[df['STUDENT_CD']==name].reset_index(drop=True)
    gpa = 0
    if tp['GPA'][0] == tp['GPA'][0] : 
        gpa = tp['GPA'][0]
        total_gpa.append(gpa)
    else : 
        total_gpa.append(gpa)
    for i in range(1, len(tp)) : 
        if tp['GPA'][i] != tp['GPA'][i] : 
            total_gpa.append(gpa)
        else : 
            gpa = (gpa + tp['GPA'][i])/2
            total_gpa.append(gpa)
            
df['TOTAL_GPA'] = total_gpa
df['TOTAL_GPA'] = df['TOTAL_GPA'].round(2)

In [ ]:
# icap 데이터 구축하는 함수 -> 내부 전역변수 선언이 있어 위치 변동 불가
def en_icap(data, date, columns) : 
    en_pau = data.copy()
    en_pau.dropna(subset=date, inplace=True)
    en_pau = en_pau.reset_index(drop=True)

    year_list = []
    semester_list = []

    for i in range(len(en_pau)) : 
        year = int(en_pau[date][i][0:4])
        month = int(en_pau[date][i][5:7])
        year_list.append(year)
        if month >= 3 and month <=8 : 
            semester_list.append(1)
        else : 
            semester_list.append(2)

    en_pau['CHG_YEAR'] = year_list
    en_pau['CHG_SMT'] = semester_list

    en_pau = en_pau[['PID', columns, 'CHG_YEAR', 'CHG_SMT']]
    en_pau.rename(columns={'PID' : 'STUDENT_CD'}, inplace=True)

    for i in range(len(en_pau)) : 
        if len(en_pau['STUDENT_CD'][i]) != 8 : 
            en_pau = en_pau.drop(i, axis=0)

    en_pau['STUDENT_CD'] = en_pau['STUDENT_CD'].astype("int64")
    global df
    df = pd.merge(df, en_pau, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# icap
en_icap(pau, 'TESTDATE', 'ISPAU')
en_icap(jtu, 'TESTDATE', 'ISJTU')
en_icap(ncs, 'WDATE', 'ISNCS')
en_icap(hcp, 'TESTDATE', 'ISHCP')
en_icap(dlc, 'TESTDATE', 'ISDLC')

In [ ]:
df['CHG_SMT'] = df['CHG_SMT'].astype('int64')
df['STUDENT_CD'] = df['STUDENT_CD'].astype('str')

In [ ]:
# 0207 수정
name_data = name_data[['학번', '이름']]
name_data.rename(columns={'학번' : 'STUDENT_CD'}, inplace=True)
df = pd.merge(df, name_data, on = ['STUDENT_CD'], how='left')

In [ ]:
def lib_pre(data, date, number) : 
    en_data = data.copy()
    year_list = []
    smt_list = []

    for i in range(len(en_data)) : 
        year_list.append(str(en_data[date][i])[:4])
        if str(en_data[date][i])[-2:] in ['03', '04', '05' ,'06', '07', '08'] : 
            smt_list.append(1)
        else : 
            smt_list.append(2)

    en_data['CHG_YEAR'] = year_list
    en_data['CHG_SMT'] = smt_list
    en_data.drop([date], axis=1, inplace=True)
    en_data['CHG_YEAR'] = en_data['CHG_YEAR'].astype('int64')
    en_data.rename(columns={number : 'STUDENT_CD'}, inplace=True)
    return en_data

In [ ]:
# gate
en_gate = lib_pre(gate, 'ENTER_DATE', 'ID')
en_gate_new = pd.concat([lib(en_gate, 2020, 1, 'ENTER_COUNT'), lib(en_gate, 2020, 2, 'ENTER_COUNT'), lib(en_gate, 2021, 1, 'ENTER_COUNT'), 
                         lib(en_gate, 2021, 2, 'ENTER_COUNT'), lib(en_gate, 2022, 1, 'ENTER_COUNT'), lib(en_gate, 2022, 2, 'ENTER_COUNT')], axis=0)

df = pd.merge(df, en_gate_new, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# book
book.drop(['CLASS_NO'], axis=1, inplace=True)
en_book = lib_pre(book, 'CHARGE_DATE', 'MEMBER_NO')

en_book_new = pd.concat([lib(en_book, 2020, 1, 'CHARGE_CNT'), lib(en_book, 2020, 2, 'CHARGE_CNT'), lib(en_book, 2021, 1, 'CHARGE_CNT'), 
                         lib(en_book, 2021, 2, 'CHARGE_CNT'), lib(en_book, 2022, 1, 'CHARGE_CNT'), lib(en_book, 2022, 2, 'CHARGE_CNT')], axis=0)

df = pd.merge(df, en_book_new, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# room
en_room = lib_pre(room, 'CHARGE_DATE', 'MEMBER_NO')

en_room_new = pd.concat([lib(en_room, 2020, 1, 'ROOM_CHARGE_CNT'), lib(en_room, 2020, 2, 'ROOM_CHARGE_CNT'), lib(en_room, 2021, 1, 'ROOM_CHARGE_CNT'), 
                         lib(en_room, 2021, 2, 'ROOM_CHARGE_CNT'), lib(en_room, 2022, 1, 'ROOM_CHARGE_CNT'), lib(en_room, 2022, 2, 'ROOM_CHARGE_CNT')], axis=0)

df = pd.merge(df, en_room_new, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# seat
en_seat = lib_pre(seat, 'CHARGE_DATE', 'MEMBER_NO')

en_seat_new = pd.concat([lib(en_seat, 2020, 1, 'SEAT_CHARGE_CNT'), lib(en_seat, 2020, 2, 'SEAT_CHARGE_CNT'), lib(en_seat, 2021, 1, 'SEAT_CHARGE_CNT'), 
                         lib(en_seat, 2021, 2, 'SEAT_CHARGE_CNT'), lib(en_seat, 2022, 1, 'SEAT_CHARGE_CNT'), lib(en_seat, 2022, 2, 'SEAT_CHARGE_CNT')], axis=0)

df = pd.merge(df, en_seat_new, on = ['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], how='left')

In [ ]:
# 여기까지 각 데이터 연결 완료 -> 이후부터 labeling 작업
# 재학인원 값 부여
temp = df.copy()
temp = temp.sort_values(by=['STUDENT_CD', 'CHG_YEAR', 'CHG_SMT'], ascending=True).reset_index(drop=True)
for i in range(len(temp)) : 
    if (temp['CDT_APPLY'][i] == temp['CDT_APPLY'][i]) & (temp['CHG_DIV'][i] != temp['CHG_DIV'][i]) : 
        temp['CHG_DIV'][i] = '재학'

In [ ]:
# 휴학인원 재분류
temp['CHG_NM'].replace('병고휴학', '일반휴학', inplace=True)
hakj = []

for name in temp['STUDENT_CD'].unique().tolist() : 
    tp = temp[temp['STUDENT_CD']==name]
    hakj_list = []
    if tp['CHG_DIV'].tolist()[0] == '휴학' : 
        hakj_list.append(tp['CHG_NM'].tolist()[0])
    else: 
        hakj_list.append(tp['CHG_DIV'].tolist()[0])
    for i in range(1, len(tp)) : 
        if (tp['CHG_DIV'].tolist()[i] != tp['CHG_DIV'].tolist()[i]) & (hakj_list[i-1] == '군입대휴학') : 
            hakj_list.append('군입대휴학')
        elif (tp['CHG_DIV'].tolist()[i] != tp['CHG_DIV'].tolist()[i]) & (hakj_list[i-1] == '일반휴학') : 
            hakj_list.append('일반휴학')
        else : 
            if tp['CHG_DIV'].tolist()[i] == '휴학' : 
                hakj_list.append(tp['CHG_NM'].tolist()[i])
            else : 
                hakj_list.append(tp['CHG_DIV'].tolist()[i])
    hakj += hakj_list
    
temp['CHG_DIV'] = hakj

In [ ]:
# 학점정보가 없고 모든 학적이 Null값인 경우 사용하기 어려운 데이터로 파악해 제거 -> drop_idx_1
# 편입생들의 경우 학적변동이 없고 22년도(최근 1년) 데이터가 모두 결측치일 때, 졸업이라고 추정하여 해당 학기 값 제거 -> drop_idx_2
drop_idx_1 = []
drop_idx_2 = []

for name in temp['STUDENT_CD'].unique().tolist() : 
    tp = temp[temp['STUDENT_CD']==name]
    if tp['CHG_DIV'].isnull().sum() == len(tp) : 
        drop_idx_1 += tp.index.tolist()
    if tp['ENTER_TYPE'].tolist()[:1][0] == '편입' : 
        if tp['CHG_DIV'].isnull().sum() != len(tp) : 
            if (tp['CHG_DIV'].tolist()[-1:][0] != tp['CHG_DIV'].tolist()[-1:][0]) : 
                drop_idx_2 += tp.index.tolist()[-1:]
                if (tp['CHG_DIV'].tolist()[-2:-1][0] != tp['CHG_DIV'].tolist()[-2:-1][0]) :
                    drop_idx_2 += tp.index.tolist()[-2:-1]
                    
temp = temp.drop(drop_idx_1, axis=0)
temp = temp.drop(drop_idx_2, axis=0)

In [ ]:
# '위험'이라는 신규 카테고리 생성 
temp['CHG_DIV'] = temp['CHG_DIV'].fillna('위험')
temp_data = temp.copy()
temp_data = temp_data.reset_index(drop=True)

In [ ]:
# 연속된 휴학횟수 카운팅
HUHAK_list = ['군입대휴학', '일반휴학']
cnt = 0 
seq_list = [0]

for i in range(1, len(temp_data)) : 
    if temp_data['STUDENT_CD'][i] == temp_data['STUDENT_CD'][i-1] : 
        if temp_data['CHG_DIV'][i] in HUHAK_list : 
            cnt += 1
            seq_list.append(cnt)
        else : 
            cnt = 0
            seq_list.append(cnt)
    else : 
        cnt = 0
        if temp_data['CHG_DIV'][i] in HUHAK_list : 
            cnt += 1
            seq_list.append(cnt)
        else : 
            seq_list.append(cnt)
            
temp_data['seq'] = seq_list

In [ ]:
# 년도-학기를 하나의 column에 합치기 ex) 20221
yy = []
for i in range(len(temp_data)) : 
    sem = str(temp_data['CHG_YEAR'][i])+str(temp_data['CHG_SMT'][i])
    yy.append(sem)
    
temp_data['YYHAKGI'] = yy

In [ ]:
# 시점에 따른 Labeling 작업 진행
semester_list = temp_data['YYHAKGI'].unique().tolist()
dropout_list = []
seq_drop_list = []
# temp_list = [] 

for i in range(len(temp_data)) : 
    if temp_data['YYHAKGI'][i] != '20222' : 
        next_semester = semester_list[semester_list.index(temp_data['YYHAKGI'][i])+1]
        temp_frame = temp_data.loc[(temp_data['YYHAKGI']==next_semester) & (temp_data['STUDENT_CD'] == temp_data['STUDENT_CD'][i])].reset_index(drop=True)
        if len(temp_frame) == 0 : 
            dropout_list.append("NoData")
            seq_drop_list.append("NOData")
        else : 
            dropout_list.append(temp_frame['CHG_DIV'][0])
            seq_drop_list.append(temp_frame['seq'][0])
    else : 
        dropout_list.append("NoData")
        seq_drop_list.append("NoData")

temp_data['HAKJ'] = dropout_list
temp_data['seq_la'] = seq_drop_list

In [ ]:
# label 부여
label_list = []

for i in range(len(temp_data)) : 
    if temp_data['HAKJ'][i] in ['재학', '복학', '전과'] : 
        label_list.append(0)
    elif temp_data['HAKJ'][i] in ['제적', '위험'] : 
        label_list.append(1)
    elif temp_data['HAKJ'][i] == '군입대휴학' : 
        if temp_data['seq_la'][i] > 4 : #column명 수정
            label_list.append(1)
        else : 
            label_list.append(0)
    elif temp_data['HAKJ'][i] == '일반휴학' : 
        if temp_data['seq_la'][i] > 2 : #column명 수정
            label_list.append(1)
        else : label_list.append(0)
    else : 
        label_list.append("NoData")
        
temp_data['LABEL'] = label_list

In [ ]:
# 0208 수정
temp_data.drop(['seq', 'YYHAKGI', 'HAKJ', 'seq_la', 'CHG_NM', 'CHG_REA'], axis=1, inplace=True)

In [ ]:
# 결측비율 계산
missing_temp = temp_data.loc[(temp_data['CHG_YEAR']==2021) & (temp_data['CHG_SMT']==1)] 
missing_temp = missing_temp.loc[missing_temp['LABEL'] != 'NoData']
missing_rate = (missing_temp.isnull().sum()/len(missing_temp)).to_dict()

In [ ]:
en_data = temp_data.copy()

In [ ]:
# 결측치 전처리
en_data['HI_SCH_AREA'] = en_data['HI_SCH_AREA'].fillna('서울')
for i in range(len(en_data)) : 
    if en_data['GRADUATION_YEAR'][i] != en_data['GRADUATION_YEAR'][i] : 
        en_data['GRADUATION_YEAR'][i] = en_data['ENTER_YEAR'][i]
        
en_data[['MAJOR_NM', 'DOUBLE_MAJOR_NM', 'LINK_MAJOR_NM', 'SUB_MAJOR_NM']] = en_data[['MAJOR_NM', 'DOUBLE_MAJOR_NM', 'LINK_MAJOR_NM', 'SUB_MAJOR_NM']].fillna('없음')

In [ ]:
en_data['IS_REG_STUDENT_FEE'] = en_data['IS_REG_STUDENT_FEE'].fillna('Y')
en_data['IS_REG_YET'] = en_data['IS_REG_YET'].fillna('Y')
en_data['IS_REG_DIV'] = en_data['IS_REG_DIV'].fillna('N')
en_data['CONFIRM_YN'] = en_data['CONFIRM_YN'].fillna('N')
en_data['CONTINUE_YN'] = en_data['CONTINUE_YN'].fillna('N')
en_data['WORK_YN'] = en_data['WORK_YN'].fillna('N')
en_data['WARN_YN'] = en_data['WARN_YN'].fillna('N')
en_data['ISPAU'] = en_data['ISPAU'].fillna('N')
en_data['ISJTU'] = en_data['ISJTU'].fillna('N')
en_data['ISNCS'] = en_data['ISNCS'].fillna('N')
en_data['ISHCP'] = en_data['ISHCP'].fillna('N')
en_data['ISDLC'] = en_data['ISDLC'].fillna('N')

In [ ]:
en_data = en_data.fillna(0)

In [ ]:
train_data = en_data.loc[(en_data['CHG_YEAR']==2021) & (en_data['CHG_SMT']==1)]
inf_data = en_data.loc[(en_data['CHG_YEAR']==2022) & (en_data['CHG_SMT']==2)]

In [ ]:
train_data = train_data.loc[train_data['LABEL'] != 'NoData']
inf_data.drop(['LABEL'], axis=1, inplace=True)

In [ ]:
# 0208 추가
inf_data = inf_data.reset_index(drop=True)
for i in range(len(inf_data)) : 
    if inf_data['CHG_DIV'][i] in ['제적'] : 
        inf_data.drop([i], axis=0, inplace=True)

In [ ]:
# 0208 최종
train_data['CHG_DIV'].replace({'복학' : '재학',  '전과' : '재학', '위험' : '재학'}, inplace=True)
inf_data['CHG_DIV'].replace({'복학' : '재학', '전과' : '재학', '위험' : '재학'}, inplace=True)

In [ ]:
name_dict = {}
for i in range(len(column_name)) : 
    name_dict[column_name['feature'][i]] = column_name['feature_name'][i]

In [ ]:
# train_data, inference_data 분리, missing_rate pickle 저장
train_data.to_csv('/ml/algorithmlabs.inference/train_data.csv', index=False)
inf_data.to_csv('/ml/algorithmlabs.inference/inference_data.csv', index=False)
joblib.dump(missing_rate, '/ml/algorithmlabs.inference/pickle/missing_rate.pkl')
joblib.dump(name_dict, '/ml/algorithmlabs.inference/pickle/name_dict.pkl')